In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 定义超参数
batch_size = 64
learning_rate = 0.01
num_epochs = 10

# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载训练集和测试集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 创建数据加载器
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 定义模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

# 初始化模型、损失函数和优化器
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 训练模型
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# 测试模型
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# 运行训练和测试
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301688
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.183030
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.284493
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.504701
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.222532
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.435761
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.172196
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.196458
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.286420
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.210827

Test set: Average loss: 0.0030, Accuracy: 9456/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.187740
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.110141
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.087697
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.177602
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.132348
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.186107
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.089049
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.207795
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.198051
T

In [2]:
import torch
from torchvision import transforms
from PIL import Image

# 加载模型
model = Net()
model.load_state_dict(torch.load('model.pth'))
model.eval()

# 预处理图片
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载图片
image = Image.open("C:\\Users\\18026\\Desktop\\b48be30c49453f8044da5a2835d12167.png").convert('L')  # 转换为灰度图
image = transform(image).unsqueeze(0)  # 添加批次维度

# 运行模型
with torch.no_grad():
    output = model(image)

# 解析输出
_, predicted = torch.max(output.data, 1)
print(f'Predicted digit: {predicted.item()}')


C:\Users\18026\AppData\Local\Temp\ipykernel_3200\1749054576.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth'))


FileNotFoundError: [Errno 2] No such file or directory: 'model.pth'